In [12]:
!python3 --version


Python 3.10.12


In [15]:
from PIL import Image
print(Image.__version__)
#11.0.0

11.0.0


In [5]:
import rembg
print(rembg.__version__)

2.0.60


In [7]:
import onnxruntime
print(onnxruntime.__version__)

1.20.1


In [13]:
#Python 3.10.12

!pip install rembg==2.0.60 onnxruntime==1.20.1 Pillow==11.0.0

!apt-get install -y fonts-roboto  # Install Roboto font

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-roboto is already the newest version (2:0~20170802-3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


# Code for Text Ovellay

In [14]:


# Import libraries
from rembg import remove
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive
from google.colab import files
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create paths
input_folder = '/content/drive/My Drive/background_removal'
output_folder = '/content/drive/My Drive/background_removal/output'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

def add_text_to_image(image, text, position=None, font_size=120, text_color=(255, 255, 255, 255)):
    """Add text to an image with enhanced styling"""
    # Create a transparent overlay for the text
    text_overlay = Image.new('RGBA', image.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(text_overlay)

    try:
        # Try different font options
        try:
            # Try to use Roboto Bold
            font = ImageFont.truetype("Roboto-Bold.ttf", font_size)
        except:
            try:
                # Try system Arial Bold
                font = ImageFont.truetype("Arial Bold.ttf", font_size)
            except:
                # Fallback to DejaVu Sans Bold
                font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
    except:
        # Last resort fallback
        font = ImageFont.load_default()

    # Calculate text size
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # If position is not specified, center the text
    if position is None:
        position = ((image.width - text_width) // 2, (image.height - text_height) // 2)

    # Add shadow effect (optional)
    shadow_color = (0, 0, 0, 180)  # Semi-transparent black
    shadow_offset = 3
    draw.text((position[0] + shadow_offset, position[1] + shadow_offset),
              text, font=font, fill=shadow_color)

    # Add main text
    draw.text(position, text, font=font, fill=text_color)

    return text_overlay

def create_layered_image(original_image, text, removed_bg_image, text_position=None):
    """Create image with three layers: original image, text, and PNG overlay"""
    # Ensure all images are in RGBA mode
    if original_image.mode != 'RGBA':
        original_image = original_image.convert('RGBA')
    if removed_bg_image.mode != 'RGBA':
        removed_bg_image = removed_bg_image.convert('RGBA')

    # Create text overlay with larger text
    text_overlay = add_text_to_image(
        original_image,
        text,
        position=text_position,
        font_size=200,  # Larger font size
        text_color=(255, 255, 255, 255)  # Solid white with full opacity
    )

    # Resize removed background image if needed
    if removed_bg_image.size != original_image.size:
        removed_bg_image = removed_bg_image.resize(original_image.size, Image.LANCZOS)

    # Layer 1: Original image (already set as base)
    result = original_image

    # Layer 2: Add text overlay
    result = Image.alpha_composite(result, text_overlay)

    # Layer 3: Add removed background image on top
    result = Image.alpha_composite(result, removed_bg_image)

    return result

def process_from_drive(image_name, text="PARTIAL\nWORLD"):
    input_path = os.path.join(input_folder, image_name)
    final_output_path = os.path.join(output_folder, 'layered_' + os.path.splitext(image_name)[0] + '.png')

    # Open original image
    original_image = Image.open(input_path).convert('RGBA')

    # Create removed background version
    removed_bg = remove(original_image)

    # Create final layered image
    final_image = create_layered_image(original_image, text, removed_bg)

    # Save final result
    final_image.save(final_output_path, format='PNG')
    print(f"Saved to: {final_output_path}")

def process_uploaded_image(text="PARTIAL\nWORLD"):
    print("Please select an image to upload:")
    uploaded = files.upload()
    filename = next(iter(uploaded))

    # Open original image
    original_image = Image.open(filename).convert('RGBA')

    # Create removed background version
    removed_bg = remove(original_image)

    # Create final layered image
    final_image = create_layered_image(original_image, text, removed_bg)

    # Save and download result
    final_output_path = os.path.join(output_folder, 'layered_' + os.path.splitext(filename)[0] + '.png')
    final_image.save(final_output_path, format='PNG')
    print(f"Saved to: {final_output_path}")

    # Download the result
    files.download(final_output_path)

# Choose your preferred method:
# 1. For image in Drive:
# process_from_drive('your_image.jpg', text='PARTIAL\nWORLD')

# 2. For uploading new image:
process_uploaded_image(text='DANISH\nAMMAR')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Please select an image to upload:


Saving 1726067145910d9juwbc7.png to 1726067145910d9juwbc7.png
Saved to: /content/drive/My Drive/background_removal/output/layered_1726067145910d9juwbc7.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Code for Backgroung Removal

In [ ]:


# Import libraries
from rembg import remove
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive
from google.colab import files
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create paths
input_folder = '/content/drive/My Drive/background_removal'
output_folder = '/content/drive/My Drive/background_removal/output'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

def add_text_to_image(image, text, position=None, font_size=120, text_color=(255, 255, 255, 255)):
    """Add text to an image with enhanced styling"""
    # Create a transparent overlay for the text
    text_overlay = Image.new('RGBA', image.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(text_overlay)

    try:
        # Try different font options
        try:
            # Try to use Roboto Bold
            font = ImageFont.truetype("Roboto-Bold.ttf", font_size)
        except:
            try:
                # Try system Arial Bold
                font = ImageFont.truetype("Arial Bold.ttf", font_size)
            except:
                # Fallback to DejaVu Sans Bold
                font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
    except:
        # Last resort fallback
        font = ImageFont.load_default()

    # Calculate text size
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # If position is not specified, center the text
    if position is None:
        position = ((image.width - text_width) // 2, (image.height - text_height) // 2)

    # Add shadow effect (optional)
    shadow_color = (0, 0, 0, 180)  # Semi-transparent black
    shadow_offset = 3
    draw.text((position[0] + shadow_offset, position[1] + shadow_offset),
              text, font=font, fill=shadow_color)

    # Add main text
    draw.text(position, text, font=font, fill=text_color)

    return text_overlay

def create_layered_image(original_image, text, removed_bg_image, text_position=None):
    """Create image with three layers: original image, text, and PNG overlay"""
    # Ensure all images are in RGBA mode
    if original_image.mode != 'RGBA':
        original_image = original_image.convert('RGBA')
    if removed_bg_image.mode != 'RGBA':
        removed_bg_image = removed_bg_image.convert('RGBA')

    # Create text overlay with larger text
    text_overlay = add_text_to_image(
        original_image,
        text,
        position=text_position,
        font_size=200,  # Larger font size
        text_color=(255, 255, 255, 255)  # Solid white with full opacity
    )

    # Resize removed background image if needed
    if removed_bg_image.size != original_image.size:
        removed_bg_image = removed_bg_image.resize(original_image.size, Image.LANCZOS)

    # Layer 1: Original image (already set as base)
    result = original_image

    # Layer 2: Add text overlay
    result = Image.alpha_composite(result, text_overlay)

    # Layer 3: Add removed background image on top
    result = Image.alpha_composite(result, removed_bg_image)

    return result

def process_from_drive(image_name, text="PARTIAL\nWORLD"):
    input_path = os.path.join(input_folder, image_name)
    final_output_path = os.path.join(output_folder, 'layered_' + os.path.splitext(image_name)[0] + '.png')

    # Open original image
    original_image = Image.open(input_path).convert('RGBA')

    # Create removed background version
    removed_bg = remove(original_image)

    # Create final layered image
    final_image = create_layered_image(original_image, text, removed_bg)

    # Save final result
    final_image.save(final_output_path, format='PNG')
    print(f"Saved to: {final_output_path}")

def process_uploaded_image(text="PARTIAL\nWORLD"):
    print("Please select an image to upload:")
    uploaded = files.upload()
    filename = next(iter(uploaded))

    # Open original image
    original_image = Image.open(filename).convert('RGBA')

    # Create removed background version
    removed_bg = remove(original_image)

    # Create final layered image
    final_image = create_layered_image(original_image, text, removed_bg)

    # Save and download result
    final_output_path = os.path.join(output_folder, 'layered_' + os.path.splitext(filename)[0] + '.png')
    final_image.save(final_output_path, format='PNG')
    print(f"Saved to: {final_output_path}")

    # Download the result
    files.download(final_output_path)

# Choose your preferred method:
# 1. For image in Drive:
# process_from_drive('your_image.jpg', text='PARTIAL\nWORLD')

# 2. For uploading new image:
process_uploaded_image(text='DANISH\nAMMAR')